# Function to calculate the probability of correlations between two arrays
**Not Able to run on it's own**
### Variables to be set by the calling file:
max_shift:
Determines what array shifts are allowed:
The shifts represent the time delays in days;
The time delay with the highest probability of correlation represents
the number of days after which the numbers of one county show a similar development
as the numbers of another county.

## Modules
Needed to use non-Python functionalities already programmed by someone else.

In [1]:
# Used to cast the polygons into np.arrays and afterwards transpose them easily with the .T function
import numpy as np
import matplotlib.pyplot as plt    # to plot the counties
import math    # to calculate the square root to scale

## Control
Set variables to "True" to trigger the action described by the comment and the variable's name.

In [2]:
test_get_scaled_correlation_with_sinus = False

## Define correlation function
Contains getting the correlations between the two given arrays, scaling the correlations by the number of products, returning the correlation, the shift between the two arrays, the number of products and the highest correlation.
<br/><br/>
The variable max_shift determines the maximum shift of array2 regarding array one. if it is a negative value, all possible shifts are done.

In [3]:
def get_scaled_correlation(input_array1, input_array2, max_shift=-1):
    autocorrelation_a1 = np.correlate(input_array1,input_array1)
    array1 = [e/math.sqrt(autocorrelation_a1) for e in input_array1]
    autocorrelation_a2 = np.correlate(input_array2,input_array2)
    array2 = [e/math.sqrt(autocorrelation_a2) for e in input_array2]
    if len(array1) != len(array2):
        print("The given arrays have different lengths, this might cause problems.")

    # create array with the probabilities for a correlation between
    # array1 and array2 for every shift possible
    correlations = np.correlate(array1, array2, mode="full")

    # Get the number of products
    min_len = min(len(array1), len(array2))
    number_of_products = (# start of array2 left from start of array1
                          [e for e in range(1, min_len)] +
                          [min_len for e in range(abs(len(array2)-len(array1)), 0)] +
                          # end of array2 right from end of array1
                          [e for e in range(min_len, 0, -1)])

    # Generate the array containing the shift of array2 relative to array1
    position_array2_to_array1 = ([e for e in range(-len(array2) + 1, 0)] +
                                 [e for e in range(0, len(array1))])

    if (len(correlations) != len(position_array2_to_array1) or
        len(correlations) != len(number_of_products)):
        raise Exception("The output arrays of the get_scaled_correlation-function " +
                        "do not have the same length.")

    if max_shift < 0:
        # Scale by dividing by the number of products
        correlations = correlations // number_of_products
        return (list(correlations)#,
                #list(number_of_products),
                #list(position_array2_to_array1), 
                #int(position_array2_to_array1[np.argmax(correlations)])
               )
    # Scale by dividing by the number of products
    correlations = correlations // number_of_products
    return (#list(correlations),
            list(number_of_products),
            list(position_array2_to_array1), 
            #int(position_array2_to_array1[np.argmax(correlations)])
    )
                
    
                
                
                
def unused_old_output_of_get_scaled_correlation():
    borders = (np.where(position_array2_to_array1 == -max_shift)[0][0], 
               np.where(position_array2_to_array1 == max_shift)[0][0] + 1)
    # Scale by dividing by the number of products
    correlations = (correlations[borders[0]:borders[1]] //
                    number_of_products[borders[0]:borders[1]])
    return(
        list(correlations),
        list(number_of_products[borders[0]:borders[1]]),
        list(position_array2_to_array1[borders[0]:borders[1]]),
        int(position_array2_to_array1[np.argmax(correlations) + borders[0]])
    )

## Test the correlation function with a simple sinus

In [4]:
if test_get_scaled_correlation_with_sinus:
    sinus = [int(100*(e+1)) for e in np.sin(np.linspace(0,np.pi*10,1000))]
    plt.plot(sinus)
    print(get_scaled_correlation(sinus, sinus)[3])
    print("(should be 0. In this case shifting by -1 and 1 produces bigger correlation " +
          "probabilities, because the discrete values of the sinus.)")
    print(get_scaled_correlation(sinus, sinus)[0])
    print(get_scaled_correlation(sinus, sinus)[1])
    print(get_scaled_correlation(sinus, sinus)[2])
    plt.plot(get_scaled_correlation(sinus, sinus)[0])

## Calculate the Correlation for all Counties

In [5]:
non_county_specific_data["correlation_number_of_products"] = get_scaled_correlation(
    covid19["1001"]["incidences"], covid19["1001"]["incidences"], max_shift=100)[0]
non_county_specific_data["correlation_position_array2_to_array1"] = get_scaled_correlation(
    covid19["1001"]["incidences"], covid19["1001"]["incidences"], max_shift=100)[1]
length_of_correlation = len(non_county_specific_data["correlation_number_of_products"])
for AdmUnitID, county in covid19.items():
    county["correlation"] = dict()
    for second_AdmUnitID, second_county in covid19.items():
        county["correlation"][second_AdmUnitID] = get_scaled_correlation(
            county["incidences"], second_county["incidences"])
        if len(county["correlation"][second_AdmUnitID]) != length_of_correlation:
            raise Exception("The correlation between " + AdmUnitID + " and " +
                            second_AdmUnitID + " has " +
                            str(len(county["correlation"][second_AdmUnitID])) +
                            " elements instead of " +
                            str(length_of_correlation) + ".")

## Calculate the Correlation for all Districts

In [6]:
for districtID, district in districts.items():
    district["correlation"] = dict()
    for second_districtID, second_district in districts.items():
        district["correlation"][second_districtID] = get_scaled_correlation(
            district["incidences"], second_district["incidences"])
        if len(district["correlation"][second_districtID]) != length_of_correlation:
            raise Exception("The correlation between " + districtID + " and " +
                            second_districtID + " has " +
                            str(len(district["correlation"][second_districtID])) +
                            " elements instead of " +
                            str(length_of_correlation) + ".")